In [1]:
import os
import email 
import re
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')
res_path="C:\\Users\\vrajp\\Downloads\\COMP-5013-dataset2\\dataset2\\train" 

In [2]:
resources=os.listdir(res_path)

In [3]:
electronic_dataset=os.listdir(res_path+"\\"+resources[1])
hardware_dataset=os.listdir(res_path+"\\"+resources[0])

In [4]:
def prepare_dataset(res_path,res_name,dataset):
    final_string=""
    for i in range(len(dataset)):
        
        with open(res_path+"\\"+res_name+"\\"+dataset[i]) as f:
            file_text=f.read()
            
            message = email.message_from_string(file_text)
            data=str(message.get_payload(decode=True))
            data=data.replace('\\n',"")
            file = re.sub(r'\W', ' ', data)
            file = re.sub(r'\s+', ' ', file)
            file=re.sub('(\s+)(I|a|an|am|and|the)(\s+)', '', file)
            file=re.sub(r'[0-9]','',file)
            file=file.replace('_','')
            final_string+=" "+file
            final_string=" ".join(final_string.split())
    tokens = word_tokenize(final_string)
    for i in tokens:
        if(len(i) > 1):
            continue
        else:
            tokens.remove(i)
    return tokens

In [5]:
def generate_word_frequency(dataset):
    final_dictionary=dict()
    for i in range(len(dataset)):
        if dataset[i] in final_dictionary.keys():
            continue
        else:
            final_dictionary[dataset[i]]=dataset.count(dataset[i])
    final_dictionary=sorted(final_dictionary.items())
    return dict(final_dictionary)

In [6]:
def generate_probabilityOfWord(dataset):
    length=len(dataset)
    for i in dataset.keys():
        dataset.update({i:float('%.7f' % float(dataset[i]/length))})
    return dataset   

In [7]:
%%time
electronic_final_dataset=prepare_dataset(res_path,resources[1],electronic_dataset)

electronic_word_frequency=generate_word_frequency(electronic_final_dataset)

hardware_final_dataset=prepare_dataset(res_path,resources[0],hardware_dataset)

hardware_word_frequency=generate_word_frequency(hardware_final_dataset)

electronic_probability=generate_probabilityOfWord(electronic_word_frequency)

hardware_probability=generate_probabilityOfWord(hardware_word_frequency)


CPU times: total: 21.8 s
Wall time: 28.2 s


In [8]:
def prepare_testdataset(res_path,res_name,dataset):
    final_string=[]
    for i in range(len(dataset)):
        
        with open(res_path+"\\"+res_name+"\\"+dataset[i]) as f:
            file_text=f.read()
            
            message = email.message_from_string(file_text)
            data=str(message.get_payload(decode=True))
            data=data.replace('\\n',"")
            file = re.sub(r'\W', ' ', data)
            file = re.sub(r'\s+', ' ', file)
            file=re.sub('(\s+)(I|a|an|am|and|the)(\s+)', '', file)
            file=re.sub(r'[0-9]','',file)
            file=file.replace('_','')
            file=" "+file
            file=" ".join(file.split())
            tokens = word_tokenize(file)
            for i in tokens:
                if(len(i) > 1):
                    continue
                else:
                    tokens.remove(i)
            final_string.append(tokens)
    return final_string

In [9]:
def naive_bayes_classifier(electronic_probability,hardware_probability,res):
    result=[]
    for k in res:
        test_dataset=os.listdir(test_path+"\\"+k)
        dataset=prepare_testdataset(test_path,k,test_dataset)
        for i in dataset:
            elect_probability=1.0
            hard_probability=1.0
            for j in i:
                if j in electronic_probability.keys():
                    elect_probability=elect_probability*electronic_probability[j]
                if j in hardware_probability.keys():
                    hard_probability=elect_probability*hardware_probability[j]
            if (hard_probability>elect_probability):
                    result.append(['comp.sys.ibm.pc.hardware',k])
            if (hard_probability<elect_probability): 
                    result.append(['sci.electronics',k])
    return result

In [10]:
def calculate_accuracy(data):
    correct=0
    for i in range(len(data)):
        if(data[i][0]==data[i][1]):
            correct+=1
    return float(correct/len(data))*100       

In [11]:
test_path="C:\\Users\\vrajp\\Downloads\\COMP-5013-dataset2\\dataset2\\test" 
test_resources=os.listdir(test_path)

In [12]:
result=naive_bayes_classifier(electronic_probability,hardware_probability,test_resources)

In [13]:
Accuracy=calculate_accuracy(result)
print("Accuracy===>",Accuracy)

Accuracy===> 43.44827586206896
